[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CCS-ZCU/pribehy-dat/blob/master/scripts/regex.ipynb)

Tento soubor je součástí sestavy elektronických studijních opor [Příběhy dat: Výpočetní přístupy ke studiu kultury a společnosti](https://github.com/CCS-ZCU/pribehy-dat/tree/master). 


# Regulární výrazy

**autor**: *Vojtěch Kaše* (kase@ff.zcu.cz)

[![](https://ccs.zcu.cz/wp-content/uploads/2021/10/cropped-ccs-logo_black_space_240x240.png)](https://ccs.zcu.cz)

## Úvod a cíle kapitoly

V této kapitole si ukážeme základy tzv. regulárních výrazů. Regulární výrazy představují velice sofistikovaný způsob jak vyhledávat v textových datech určité řetězce znaků a případně je nahrazovat jinými řetězci znaků. Na rozdíl od standardního vyhledávání však regulární výrazy umožňují vyhledávání na abstraktnější rovině, pomocí obecnějších pravidel či "regulí". K tomu se používají speciálně nadefinované pomocné znaky. Regex je technologie, která nemá co dočinění s Pythonem - lze jej používat v mnoha různých aplikacích, včetně webových (např. https://regex101.com). 

V rámci Pythonu je pro regex dostupná velice praktická knihovna `re`, na které budou některé ukázky níže postaveny. Pro systematičtější pohled však tentokrát z našeho juypter notebooku na chvíli vystoupíme a projedeme si sérii cvičeních na webové stránce https://regexone.com.

Knihovna `re` v sobě má celou řadu užitečných metod. My zde budeme pracovat jen s několika z nich. Zejména
  * `re.search(pattern, string)`: nalezne uvnitř prohledávaného řetězce znaků (parametr `string`) první výskyt (tzv. `match`) hledáného řetězce znaků (`pattern`) a poskytne nám informaci o tom, kde se hledaný řetězec nachází
  * `re.findall(pattern, string)`:  nalezne uvnitř prohledávaného řetězce znaků (parametr `string`) všechny výskyty (`matches`) hledáného řetězce znaků (`pattern`) a poskytne nám informaci o tom, kde se hledané řetězce nachází
  * `re.sub(pattern, repl, string)`:  nalezne uvnitř prohledávaného řetězce znaků (parametr `string`) všechny výskyty hledáného řetězce znaků (`pattern`) a nahradí jej řetězcem `repl`.


In [ ]:
import requests
import re
import pandas as pd
import matplotlib.pyplot as plt
!pip install lxml
from bs4 import BeautifulSoup
from getpass import getpass
import json

Představte si, že chcete v dokumentu vyhledat veškéré výskyty číslic. Pomocí regexu toho lze dosáhnout pomocí použití speciálního znaku "\d".

In [ ]:
# náš dokument se nachází uvnitř proměnné text
text = "Tento text obsahuje kromě znaků pro písmena i některé číslice, jako je třeba 7, 13, 542, nebo letopočet 1842 apod. Kromě toho obsahuje i některá vlastní jména, jako je třeba Jan Patočka nebo Václav Havel. To nám stačí, abychom si zde vyzkoušeli několik základních principů regexu."

Použijeme tedy metodu `re.findall()` a speciální znak "\d"

In [ ]:
matches = re.findall("\d", text)
matches

Nyní jsme zachytili každou číslici zvlášť. Abychom zachytili i víceciferná čísla, musíme použít kvantifikátoru. V našem patternu uděláme jednu drobnou úpravu, a totiž přidáme znaménko +.

In [ ]:
matches = re.findall("\d+", text)
matches

někdy nás třeba mohou zajímat pouze čísla o určitém počtu cifer, jako jsou letopočty, které jsou již více než tisíc let čtyřciferné a nejspíš ještě několik tisíc let budou. Toho lze dosáhnout kvantifikátorem `{}`, 

In [ ]:
matches = re.findall("\d{4}", text)
matches

Detekce čísel je jen jeden možný příklad. Druhý příklad, než se pustíme do reálných dat, budou vlastní jména. Vlastní jména začínají velkými písmeny. Najděme tedy všechna slova začínající velkými písmeny:

In [ ]:
re.findall("[A-Z]\w*", text)

Ano, je zde několik problému. Zaprvé zachycujeme také všechny slova na začátku vět. Jak tomu předejít? To přenechám samostatnému úkolu:

In [ ]:
# SAMOSTATNÝ ÚKOL:
# upravit regex níže tak, aby nezachycoval slova, která mají velká písmena proto, že se nacházejí na začátku věty. 
re.findall("[A-Z]\w*", text)

Jména v našem textu sestávají ze dvou částí: křesťního jména a příjmení. Co se tedy soustředit pouze na situace, kdy se jedná o dvě slova s velkými písmeny po sobě:

In [ ]:
re.findall("[A-Z]\w*\\s[A-Z]\w*", text)

## Regulérní výrazy a metadata ze Slovníku českých filozofů

Nyní si opět načteme data ze slovníku českých filozofů, která jsme vydolovali v jedné z předchozích kapitol.

In [ ]:
slovnik_df = pd.read_csv("https://raw.githubusercontent.com/CCS-ZCU/pribehy-dat/master/data/slovnik_df.csv")
slovnik_df.head(5)

Ve sloupcích narození a smrt vidíme, že se zde datum a místo narození objevují vždy ve víceméně shodné podobě. Zajímá-li nás např. pouze rok narození či úmrtí, mohlo by nám postačit vyhledat v každém řetězci čtyřciferná čísla. Objevují se zde však i kompletní datumy a místní názvy, kde k těmto událostem došlo.

Uvědomme si, že zde pracujeme  s objektem typu dataframe knihovny pandas. Některé věci je proto potřeba dělat trochu odlišně než na příkladech výše. Naše regexy si zabalíme do vlastních definovaných funkcí, které následně budeme aplikovat na všechny hodnoty ve vybraném sloupci najednou. Funkci níže budeme postupně upravovat tak, aby zachytila více a více zajímavýcj informací. Výhoda této funkce je, že v sobě má "pojistku" ve formě výjimku (try-except) pro situace, že se hledání nepovede, například z důvodu nevalidních vstupních dat - měli bychom se tedy vyhnout mnoha zbytečným chybovým hláškám.

In [ ]:
def our_regex_matcher(string, regex_pattern):
    try:
        return re.search(regex_pattern, string).group() 
    except:
        return None

V této základní podobě funkce zachytí pouze čtyřciferné číslo, což v našem případě bude typicky letopočet: Funkci můžeme otestovat na hned první hodnotě ve sloupci "birth":

In [ ]:
our_string = "* 31. 8. 1546 Veleslavín u Prahy"
our_regex_matcher(our_string, "\d{4}")

Nyní tuto funkci aplikujeme na všechny hodnoty v celém sloupci "birth" a "death" a výsledek přiřadíme dvěma novým sloupcům: "birth_year" a "death_year" 

In [ ]:
slovnik_df["birth_year"] = slovnik_df['birth'].apply(our_regex_matcher, args=("\d{4}",))
slovnik_df["death_year"] = slovnik_df['death'].apply(our_regex_matcher, args=("\d{4}",))  # 
# vypíšeme si prvních 10 řádek dataframu"
slovnik_df.head(10)


Ač to na první pohlede není zřejmé, naše sloupce s letopočty obsahují ve skutečnosti proměnné typu string. Takže si je ještě převedeme na čísla.

In [ ]:
slovnik_df["birth_year"] = slovnik_df["birth_year"].apply(lambda x: int(x) if x != None else None)
slovnik_df["death_year"] = slovnik_df["death_year"].apply(lambda x: int(x) if x != None else None)

Takto naformátované letopočty již umožňují časové temporální analýzy. Například se můžeme podívat na distribuci narození od 18. století do současnosti po dekádách. 

In [ ]:
fig, ax = plt.subplots()
slovnik_df["birth_year"].hist(bins=range(1800, 2000, 10), ax=ax)
ax.set_xlim(1801,2000)

Vidíme zde, že nezanedbatelný počet filozofů zahrnutých ve slovníku se narodil ve 20. letech 20. století. To nemusí nutně znamenat, že by se v této dekádě rodilo více filozofických talentů, ale může to být dáno zaměřením autorů SLovníku, kteří věnovali zvláštní pozornost osobnostem své a bezprostředně předcházející generace. 

Ve sloupcích "birth" a "death" se ve většině případů nachází i informace o tom, kde k těmto událostem došlo. Tato informace se nachází bezprostředně za letopočtem a mezerou. Opět ji můžeme vyxetrahovat pomocí regexu. Vytvoříme si proto však samostatnou funkci:

In [ ]:
def get_place(string, regex_pattern="(\d{4})(\s)(.+)"):
    try:
        return re.search(regex_pattern, string).groups()[2]
    except:
        return None

In [ ]:
our_string = "* 31. 8. 1546 Veleslavín u Prahy"
get_place(our_string)

In [ ]:
slovnik_df["birth_place"] = slovnik_df["birth"].apply(get_place)
slovnik_df["death_place"] = slovnik_df["death"].apply(get_place)
slovnik_df.head(5)

Máme-li místa narození a úmrtí takto v samostatné podobě, můžeme se pro přehled opět podívat na jejich distribuci:

In [ ]:
slovnik_df["birth_place"].value_counts()

In [ ]:
slovnik_df["death_place"].value_counts()

# Rozšiřující analýza: Automatické geokódováníí míst narození a úmrtí  

Prostorovou informaci nyní máme pouze v podobě textu. Tento formát nám však neumožňuje studovat prostorové distribuce a vztahy pomocí metod **GIS** (viz samostatná kapitola na toto téma, kde se k těmto datům možná vrátíme.

Abychom s našimi daty mohli takto pracovat, je třeba je tzv. geokódovat, tj. opatřit je koordináty zeměpisné šířky a délky. Toho lze dosáhnout buď *manuálně*, postupným procházením jednotlivých místních názvů a dohledávání jich v mapě, a nebo *automaticky*. Pro automatické vyhledávání koordinátů pro místní názvy se nabízí platforma https://www.geonames.org, která disponuje i jednoduchým **API** (viz kapitola o API).

Nejprve si vytvořme si seznam všech unikátních míst v obou sloupcích:

In [ ]:
places_unique = [el for el in set(list(slovnik_df["death_place"]) + list(slovnik_df["birth_place"])) if el != None]
len(places_unique)

Vidíme, že máme co do činění s 584 unikátními místními názvy. Pro zkoušku si vypíšeme 10 z nich:

In [ ]:
places_unique[:10]

Nyní můžeme navštívit webovou stránku https://www.geonames.org a vytvořit si na ni účet. Účet si aktivujeme a především pro něj umožňíme používání "free webservices" (jinak nám api nebude fungovat). 

(V případě, že si např. z důvodu časové tísně nechceme účet vytvořit, následující tři buňky přeskočíme a načteme si již mnou předem získaná data.)

In [ ]:
# zadáme naše uživatelské jméno
user = getpass("Enter your geonames username: ")

Níže jako hodnotu proměnné `place_string` na zkooušku zadáme název náhodně zvoleného místa. V případě, že vše funguje správně (tj. zejména že máme funkční a aktivovaný geonames účet a že jsme zvolili místní název, se kterým si platforma geonames umí poradit), buňka nám vypíše geografické koordináty:

In [ ]:

place_string = "Plzeň"
resp = requests.get("http://api.geonames.org/search?q={0}&username={1}".format(place_string, user))
soup = BeautifulSoup(resp.content, "xml")
lat, lng = soup.geoname.lat.string, soup.geoname.lng.string
print(lat, lng)

Stejnou postup můžeme nyní aplikovat na všech 584 unikátních míst ze Slovníku. Jinými slovy, vzneseme 584 HTTP dotazů a v datech odpovědi se pokusíme identifikovat údaje o zeměpisné šířce a délce. Tato data budeme postupně přidávat do objektu `places_unique_dict` typu `dict` - klíčem bude vždy daný zeměpisný název a hodnotou příslušné koordináty.

In [ ]:
places_unique_dict = {}
for place in places_unique:
    resp = requests.get("http://api.geonames.org/search?q={0}&username={1}".format(place.partition(" (")[0], user))
    try:
        soup = BeautifulSoup(resp.content, "xml")
        lat, lng = soup.geoname.lat.string, soup.geoname.lng.string
        places_unique_dict[place] = (lat, lng)
    except:
        pass

Pro kolik míst jsme úspěšně získali nějaké koordináty?

In [ ]:
len(places_unique_dict)

In [ ]:
# v případě, že pracujeme lokálně s repozitoří jako celkem, tak si data můžeme uložit pro další použití
#with open("../data/jsonplaces_unique_dict.json", 'w') as json_file:
#    json.dump(places_unique_dict, json_file)

Pokud jsme buňkami výše z neprošli (např. protože se nám nepodařilo aktivivovat účet, načteme si data v následující buňce tak, že odstraníme hashtag na začátku řádky a buňku spustíme.

In [ ]:
# places_unique_dict = requests.get("https://raw.githubusercontent.com/CCS-ZCU/pribehy-dat/master/data/jsonplaces_unique_dict.json").json()

Tato data nyní takříkajíc namapujeme na naše tabulková data k jednotlivým filozofům. Vytvoříme dva nové atributy: "birth_latlong" a "death_latlong".  

In [ ]:
slovnik_df["birth_latlong"] = slovnik_df["birth_place"].apply(lambda x: places_unique_dict[x] if x in places_unique_dict.keys() else None)
slovnik_df["death_latlong"] = slovnik_df["death_place"].apply(lambda x: places_unique_dict[x] if x in places_unique_dict.keys() else None)

In [ ]:
slovnik_df.head(5)

In [ ]:
# pokud pracujeme lokálně s repozitoří jako celkem, tato data si uložíme pro budoucí použití.
slovnik_df.to_json("../data/slovnik_geotemporal_df.json")

In [ ]:
slovnik_df = pd.read_json("../data/slovnik_geotemporal_df.json")
slovnik_df.head(5)